# The local unitary cluster Jastrow (LUCJ) ansatz

In this tutorial, we show how to use ffsim to simulate the local unitary cluster Jastrow (LUCJ) ansatz. We'll use it to calculate the ground state energy of an ethene molecule at a stretched bond length.

In [1]:
import pyscf
import pyscf.mcscf
import ffsim

# Build a stretched ethene molecule
bond_distance = 2.678
a = 0.5 * bond_distance
b = a + 0.5626
c = 0.9289
mol = pyscf.gto.Mole()
mol.build(
    atom=[
        ["C", (0, 0, a)],
        ["C", (0, 0, -a)],
        ["H", (0, c, b)],
        ["H", (0, -c, b)],
        ["H", (0, c, -b)],
        ["H", (0, -c, -b)],
    ],
    basis="sto-6g",
    symmetry="d2h",
)
hartree_fock = pyscf.scf.RHF(mol)
hartree_fock.kernel()

# Define active space
active_space = range(mol.nelectron // 2 - 2, mol.nelectron // 2 + 2)

# Get molecular data and molecular Hamiltonian (one- and two-body tensors)
mol_data = ffsim.MolecularData.from_scf(hartree_fock, active_space=active_space)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian

# Compute FCI energy
mol_data.run_fci()

converged SCF energy = -77.4456267643962
CASCI E = -77.6290254326717  E(CI) = -3.57322412553863  S^2 = 0.0000000



## The unitary cluster Jastrow (UCJ) ansatz

Before describing the LUCJ, we first introduce the general unitary cluster ansatz (UCJ), which has the form

$$
  \lvert \Psi \rangle = \prod_{k = 1}^L \mathcal{W_k} e^{i \mathcal{J}_k} \mathcal{W_k^\dagger} \lvert \Phi_0 \rangle
$$

where $\lvert \Phi_0 \rangle$ is a reference state, often taken as the Hartree-Fock state, each $\mathcal{W_k}$ is an [orbital rotation](./02-orbital-rotation.ipynb), and each $\mathcal{J}_k$ is a diagonal Coulomb operator of the form

$$
    \mathcal{J} = \frac12\sum_{ij,\sigma \tau} \mathbf{J}^{\sigma \tau}_{ij} n_{i,\sigma} n_{j,\tau}.
$$

Note that this expression for the diagonal Coulomb operator is more general than the one introduced in [the previous tutorial](./03-double-factorized.ipynb) because the matrices $\mathbf{J}^{\sigma \tau}$ are indexed by the spins $\sigma$ and $\tau$. In order that the operator commutes with the total spin Z operator, we enforce that $\mathbf{J}^{\alpha\alpha} = \mathbf{J}^{\beta\beta}$ and $\mathbf{J}^{\alpha\beta} = \mathbf{J}^{\beta\alpha}$. As a result, we have two sets of matrices for describing the diagonal Coulomb operators: "alpha-alpha" matrices containing coefficients for terms involving the same spin, and "alpha-beta" matrices containing coefficients for terms involving different spins.

In ffsim, the UCJ ansatz operator $\prod_{k = 1}^L \mathcal{W_k} e^{i \mathcal{J}_k} \mathcal{W_k^\dagger}$ is represented by the `UCJOperator` class, which is just a dataclass that stores the diagonal Coulomb matrices and orbital rotations. A constructor method is provided to initialize the operator from a truncated double factorization of t2 amplitudes (e.g. from CCSD or MP2).

In the code cell below, we run CCSD to get the t2 amplitudes for initializing the ansatz. We'll create an ansatz operator with 2 repetitions ($L = 2$). For our reference state, we'll use the Hartree-Fock state. Since `UCJOperator` defines a unitary effect, we can use the function `apply_unitary` to apply the ansatz operator to the reference state to obtain the ansatz state. Finally, we compute the energy of the ansatz state.

In [2]:
import numpy as np
from pyscf import cc

# Get CCSD t2 amplitudes for initializing the ansatz
ccsd = cc.CCSD(
    hartree_fock,
    frozen=[i for i in range(hartree_fock.mol.nao_nr()) if i not in active_space],
)
_, t1, t2 = ccsd.kernel()

# Construct UCJ operator
n_reps = 2
operator = ffsim.UCJOperator.from_t_amplitudes(t2, n_reps=n_reps)

# Construct the Hartree-Fock state to use as the reference state
reference_state = ffsim.hartree_fock_state(norb, nelec)

# Apply the operator to the reference state
ansatz_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)

# Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
energy = np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))
print(f"Energy at initialization: {energy}")

E(CCSD) = -77.49387212754473  E_corr = -0.04824536314851485
Energy at initialization: -77.46975600021644


To facilitate variational optimization of the ansatz, `UCJOperator` implements methods for conversion to and from a vector of real-valued parameters. The precise relation between a parameter vector and the matrices of the UCJ operator is somewhat complicated. In short, the parameter vector stores the entries of the UCJ matrices in a non-redundant way (for the orbital rotations, the parameter vector actually stores the entries of their generators.)

The following code cell shows how one can define an objective function that takes as input a parameter vector and outputs the energy of the associated ansatz state, and then optimize this objective function using `scipy.optimize.minimize`. Here, we set a small limit on the number of iterations; increase the value if you would like to run it to convergence.

In [3]:
import scipy.optimize


def fun(x):
    # Initialize the ansatz operator from the parameter vector
    operator = ffsim.UCJOperator.from_parameters(x, norb=norb, n_reps=n_reps)
    # Apply the ansatz operator to the reference state
    final_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)
    # Return the energy ⟨ψ|H|ψ⟩ of the ansatz state
    return np.real(np.vdot(final_state, hamiltonian @ final_state))


result = scipy.optimize.minimize(
    fun, x0=operator.to_parameters(), method="L-BFGS-B", options=dict(maxiter=5)
)

print(f"Number of parameters: {len(result.x)}")
print(result)

Number of parameters: 72
  message: STOP: TOTAL NO. of ITERATIONS REACHED LIMIT
  success: False
   status: 1
      fun: -77.50964466939116
        x: [-1.479e-01  1.876e-01 ...  3.824e-03  6.462e-01]
      nit: 5
      jac: [ 1.333e-03 -2.134e-03 ... -5.372e-04 -3.055e-04]
     nfev: 584
     njev: 8
 hess_inv: <72x72 LbfgsInvHessProduct with dtype=float64>


## The local unitary cluster Jastrow (LUCJ) ansatz

Implementing the $e^{i \mathcal{J}_k}$ term of the UCJ ansatz requires either all-to-all connectivity or the use of a fermionic swap network, making it challenging for noisy pre-fault-tolerant quantum processors that have limited connectivity. The idea of the *local* UCJ ansatz is to impose sparsity constraints on the $\mathbf{J}^{\alpha\alpha}$ and $\mathbf{J}^{\alpha\beta}$ matrices which allow them to be implemented in constant depth on qubit topologies with limited connectivity. The constraints are specified by a list of indices indicating which matrix entries in the upper triangle are allowed to be nonzero (since the matrices are symmetric, only the upper triangle needs to be specified).

As an example, consider a square lattice qubit topology. We can place the $\alpha$ and $\beta$ orbitals in parallel lines on the lattice, with connections between these lines forming "rungs" of a ladder shape. With this setup, orbitals with the same spin are connected with a line topology, while orbitals with different spins are connected when they share the same spatial orbital. This yields the following index constraints on the $\mathbf{J}$ matrices:

$$
\begin{align*}
\mathbf{J}^{\alpha\alpha} &: \set{(p, p+1) \; , \; p = 0, \ldots, N-2} \\
\mathbf{J}^{\alpha\beta} &: \set{(p, p) \;, \; p = 0, \ldots, N-1}
\end{align*}
$$

In other words, if the $\mathbf{J}$ matrices are nonzero only at the specified indices in the upper triangle, then the $e^{i \mathcal{J}_k}$ term can be implemented on a square topology without using any swap gates, in constant depth. Of course, imposing such constraints on the ansatz makes it less expressive, so more ansatz repetitions may be required.

In the following code cell, we demonstrate the optimization of the ansatz with these constraints imposed. We still choose to use 2 repetitions, so notice that the number of parameters in the optimization has decreased from 72 to 46.

In [4]:
alpha_alpha_indices = [(p, p + 1) for p in range(norb - 1)]
alpha_beta_indices = [(p, p) for p in range(norb)]


def fun(x):
    operator = ffsim.UCJOperator.from_parameters(
        x,
        norb=norb,
        n_reps=n_reps,
        alpha_alpha_indices=alpha_alpha_indices,
        alpha_beta_indices=alpha_beta_indices,
    )
    final_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)
    return np.real(np.vdot(final_state, hamiltonian @ final_state))


result = scipy.optimize.minimize(
    fun,
    x0=operator.to_parameters(
        alpha_alpha_indices=alpha_alpha_indices, alpha_beta_indices=alpha_beta_indices
    ),
    method="L-BFGS-B",
    options=dict(maxiter=5),
)
print(f"Number of parameters: {len(result.x)}")
print(result)

Number of parameters: 46
  message: STOP: TOTAL NO. of ITERATIONS REACHED LIMIT
  success: False
   status: 1
      fun: -77.45739668895403
        x: [-4.078e-01  8.271e-02 ... -1.694e-02  7.034e-01]
      nit: 5
      jac: [ 1.815e-03 -2.700e-05 ...  3.553e-05 -8.384e-05]
     nfev: 423
     njev: 9
 hess_inv: <46x46 LbfgsInvHessProduct with dtype=float64>


## Optimize with the linear method

ffsim includes an implementation of the ["linear method"](https://arxiv.org/abs/1412.0490v2) for optimization of a variational wavefunction. The linear method often converges faster than a standard optimization algorithm like L-BFGS-B. The interface is similar to that of `scipy.optimize.minimize`, the main difference being that instead of passing a callable that directly returns the function value to be optimized, you pass two objects: a callable that returns the wavefunction, and the Hamiltonian representing the energy to be optimized as a `LinearOperator`. The code cell below shows how to use the linear method to optimize the LUCJ ansatz from the previous example. It also shows how you can use an optional callback function to save intermediate results of the optimization.

In [5]:
from collections import defaultdict
from ffsim.optimize import minimize_linear_method


# Define function that converts a list of parameters to the corresponding state vector
def params_to_vec(x: np.ndarray) -> np.ndarray:
    operator = ffsim.UCJOperator.from_parameters(
        x,
        norb=norb,
        n_reps=n_reps,
        alpha_alpha_indices=alpha_alpha_indices,
        alpha_beta_indices=alpha_beta_indices,
    )
    return ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)


# Define a callback function used to save optimization information (this is optional)
info = defaultdict(list)


def callback(intermediate_result: scipy.optimize.OptimizeResult):
    # The callback function is called after each iteration. It accepts
    # an OptimizeResult object storing the parameters and function value at
    # the current iteration, and possibly other information
    info["x"].append(intermediate_result.x)
    info["fun"].append(intermediate_result.fun)
    if hasattr(intermediate_result, "jac"):
        info["jac"].append(intermediate_result.jac)
    if hasattr(intermediate_result, "regularization"):
        info["regularization"].append(intermediate_result.regularization)
    if hasattr(intermediate_result, "variation"):
        info["variation"].append(intermediate_result.variation)


# Optimize with the linear method
result = minimize_linear_method(
    params_to_vec,
    hamiltonian,
    x0=operator.to_parameters(
        alpha_alpha_indices=alpha_alpha_indices, alpha_beta_indices=alpha_beta_indices
    ),
    maxiter=5,
    callback=callback,
)

# Print some information
print(f"Number of parameters: {len(result.x)}")
print(result)
print()
for i, (fun, jac, regularization, variation) in enumerate(
    zip(info["fun"], info["jac"], info["regularization"], info["variation"])
):
    print(f"Iteration {i + 1}")
    print(f"    Energy: {fun}")
    print(f"    Norm of gradient: {np.linalg.norm(jac)}")
    print(f"    Regularization hyperparameter: {np.linalg.norm(regularization)}")
    print(f"    Variation hyperparameter: {np.linalg.norm(variation)}")

Number of parameters: 46
 message: Stop: Total number of iterations reached limit.
 success: False
     fun: -77.47158320347539
       x: [-6.631e-01  8.976e-02 ... -2.173e-01  9.995e-01]
     nit: 5
     jac: [ 7.259e-03 -1.181e-03 ...  3.433e-03 -5.383e-03]
    nfev: 826
    njev: 5
  nlinop: 596

Iteration 1
    Energy: -77.45724086956001
    Norm of gradient: 0.010199505442545696
    Regularization hyperparameter: 0.026890960056319083
    Variation hyperparameter: 0.9931578556353272
Iteration 2
    Energy: -77.4580971045074
    Norm of gradient: 0.0076454977282466455
    Regularization hyperparameter: 0.0006243190909239268
    Variation hyperparameter: 0.9931620654125893
Iteration 3
    Energy: -77.45812347703834
    Norm of gradient: 0.007443152735598327
    Regularization hyperparameter: 1.0505969544705465
    Variation hyperparameter: 0.9931574003267095
Iteration 4
    Energy: -77.46243342363793
    Norm of gradient: 0.02558853103196335
    Regularization hyperparameter: 0.00336